In [1]:
import CoolProp.CoolProp as CP
from CoolProp.CoolProp import PropsSI
import numpy as np
import matplotlib.pyplot as plot


In [2]:
# Dados do Compressor
fluido = 'R600a'
D = 22.5e-3           # Diâmetro do pistão [m]
e = 7.5e-3            # Excentricidade [m]
Cb = 38.5e-3          # Comprimento da biela [m]
dm = 2e-3             # Reversibilidade entre eixo e cilindro [m]
V_morto = 120e-9      # Volume morto [m]
D_o_s = 8e-3          # Diâmetro do orifício de sucção [m]
D_o_d = 6.5e-3        # Diâmetro do orifício de descarga [m]
 
f = 60  #Rotação
 
T_evap = -17          # Temperatura de evaporação [C]
T_cond = 54.4         # Temperatura de condensação [C]
T_sucçao = 32.2       # Temperatura na linha de sucção [C]

In [3]:
# Propriedades do fluido

P_evap = CP.PropsSI('P', 'T', T_evap + 273.15, 'Q', 0, fluido)
P_cond = CP.PropsSI('P', 'T', T_cond + 273.15, 'Q', 0, fluido)

rho_1 = CP.PropsSI('D', 'T', T_sucçao + 273.15, 'P', P_evap, fluido)
S_1 = CP.PropsSI('S', 'T', T_sucçao + 273.15, 'P', P_evap, fluido)
h_1 = CP.PropsSI('H', 'T', T_sucçao + 273.15, 'P', P_evap, fluido)
v_1 = 1/rho_1

S_2 = S_1
rho_2 = CP.PropsSI('D', 'S', S_2, 'P', P_cond, fluido)
h_2 = CP.PropsSI('H', 'S', S_2, 'P', P_cond, fluido)
v_2 = 1.0/rho_2


h_3 = CP.PropsSI('H', 'T', T_cond + 273.15, 'Q', 0, fluido)
rho_3 = CP.PropsSI('D', 'T', T_cond + 273.15, 'Q', 0, fluido)
S_3 = CP.PropsSI('S', 'T', T_cond + 273.15, 'Q', 0, fluido)
v_3 = 1/rho_3

h_4 = h_3
T_4 = CP.PropsSI('T', 'P', P_evap, 'H', h_4, fluido) 
 
h_5 = CP.PropsSI('H', 'T', T_evap + 273.15, 'Q', 0, fluido) # Entalpia na saída do evaporador

k = (np.log(P_cond)-np.log(P_evap))/(np.log(v_1) - np.log(v_2))

print(k, P_evap, P_cond, v_1, v_2)


1.04452333012 82101.0884554079 762002.3634941425 0.52154560698821 0.06179162810848727


In [4]:
# Avaliação da eficiência volumétrica
 
L = e*2                                       # Comprimento do pistão

V_sw = (np.pi*D**2)/4*L                       # Volume varrido
 
C = V_morto/V_sw                              # Fração de volume morto
 
eta_v = 1 + C - C*(P_cond/P_evap)**(1/k)      # Eficiência volumétrica

print(Cb-e)
 

0.031


In [5]:
# Cálculo do volume pelo tempo

theta = np.array(np.linspace(0, 2*np.pi, 720))
t = theta/(2*np.pi*f)
x = -e*np.cos(theta)+(Cb**2 -(e*np.sin(theta)-dm)**2)**0.5
C_pms = np.max(x)

x_min = np.min(x)
V_t = ((np.pi*D**2)/4)*(C_pms - x) + V_morto

In [31]:
# Equações termodinâmicas

To = T_sucçao + 273.15
Po = P_evap
A_t = np.pi*D*(x - x_min)
T_cil = 90 + 273.15
V_pis = 4*e*f
m_suc = 0.002
m_vaz = 3.9e-5
m_des = m_suc - m_vaz
R = 287.058
T = []

for i, j in enumerate(t): # i é posição na lista e j é o valor de tempo na posição i
    print(i,j, Po, To)
    
    rho_g = CP.PropsSI('D', 'P', Po, 'T', To, fluido)
    mu_g = CP.PropsSI('V', 'P', Po, 'T', To, fluido)
    h = CP.PropsSI('H', 'P', Po, 'T', To, fluido)
    h_conv = 0.7*(rho_g*V_pis*D/mu_g)**0.7
    mjhj = -(m_suc*h_1 - m_des*h - m_vaz*h)
    Cv = CP.PropsSI('CVMASS','P',Po,'T',To, fluido)
    
    A0 = (h_conv*A_t[i+1]*T_cil - h*(m_suc - m_des - m_vaz) - mjhj)
    B0 = h_conv*A_t[i+1] + R*(m_suc - m_des - m_vaz) + (R/V_t[i+1])*((V_t[i+1] - V_t[i])/(t[i+1] - t[i]))
    C0 = 1/(m_suc*Cv)
             
    T_novo = (C0*A0*(t[i+1]-t[i]) + To)/(1 + C0*B0*(t[i+1]-t[i]))
    T.append(T_novo)
    Po = rho_g*R*T_novo
    
    To = T_novo
    
    # TA DANDO PROBLEMA A PARTIR DA POSIÇAO 116 POIS A TEMPERATURA E PRESSÃO ESTÃO ABSURDAS
                                                                     
print (T)

0 0.0 82101.0884554079 305.34999999999997
1 2.31803430691e-05 171819.046328 312.171995936
2 4.63606861382e-05 367673.06914 319.84931123
3 6.95410292072e-05 823958.591733 328.326403989
4 9.27213722763e-05 49358817.382 337.108803897
5 0.000115901715345 58104973.0576 346.610438317
6 0.000139082058414 60045079.2944 356.974438479
7 0.000162262401484 61389810.0772 368.23902688
8 0.000185442744553 62739961.6562 380.422405293
9 0.000208623087622 64150235.838 393.541336582
10 0.000231803430691 65622814.134 407.613351108
11 0.00025498377376 67154745.7663 422.657390871
12 0.000278164116829 68742833.5638 438.694318371
13 0.000301344459898 70384261.2231 455.747399492
14 0.000324524802967 72076757.6753 473.842744266
15 0.000347705146036 73818696.4606 493.009684521
16 0.000370885489105 75609174.9367 513.281076477
17 0.000394065832174 77448080.0812 534.693527049
18 0.000417246175243 79336143.5298 557.287552275
19 0.000440426518312 81274987.7126 581.107684025
20 0.000463606861382 83267164.2026 606.2025

TypeError: can't convert complex to float